In [601]:
import pandas as pd
import numpy as np
import os 
import pickle
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, confusion_matrix, recall_score, accuracy_score, classification_report, precision_score

In [602]:
#Chargement du DF
data = pd.read_excel('Coeur.xlsx')
data

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,40,homme,AA,140,289,0,Normal,172,Non,0.0,Ascendant,0
1,49,femme,DNA,160,180,0,Normal,156,Non,1.0,Plat,1
2,37,homme,AA,130,283,0,ST,98,Non,0.0,Ascendant,0
3,48,femme,ASY,138,214,0,Normal,108,Oui,1.5,Plat,1
4,54,homme,DNA,150,195,0,Normal,122,Non,0.0,Ascendant,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,homme,AT,110,264,0,Normal,132,Non,1.2,Plat,1
914,68,homme,ASY,144,193,1,Normal,141,Non,3.4,Plat,1
915,57,homme,ASY,130,131,0,Normal,115,Oui,1.2,Plat,1
916,57,femme,AA,130,236,0,LVH,174,Non,0.0,Plat,1


In [603]:
# Travailler avec une copie du data Frame pour le modele d'arbre de decision pour éviter de le rechager en cas d'êrreur
df1 = data.copy()
df1

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,40,homme,AA,140,289,0,Normal,172,Non,0.0,Ascendant,0
1,49,femme,DNA,160,180,0,Normal,156,Non,1.0,Plat,1
2,37,homme,AA,130,283,0,ST,98,Non,0.0,Ascendant,0
3,48,femme,ASY,138,214,0,Normal,108,Oui,1.5,Plat,1
4,54,homme,DNA,150,195,0,Normal,122,Non,0.0,Ascendant,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,homme,AT,110,264,0,Normal,132,Non,1.2,Plat,1
914,68,homme,ASY,144,193,1,Normal,141,Non,3.4,Plat,1
915,57,homme,ASY,130,131,0,Normal,115,Oui,1.2,Plat,1
916,57,femme,AA,130,236,0,LVH,174,Non,0.0,Plat,1


In [604]:
#Afficher les informations relatives au DF
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   AGE          918 non-null    int64  
 1   SEXE         918 non-null    object 
 2   TDT          918 non-null    object 
 3   PAR          918 non-null    int64  
 4   CHOLESTEROL  918 non-null    int64  
 5   GAJ          918 non-null    int64  
 6   ECG          918 non-null    object 
 7   FCMAX        918 non-null    int64  
 8   ANGINE       918 non-null    object 
 9   DEPRESSION   918 non-null    float64
 10  PENTE        918 non-null    object 
 11  CŒUR         918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [605]:
#Normalisation des variables quantitatives
for col in df1.drop('CŒUR', axis =1).select_dtypes(np.number).columns:
    df1[col] = df1[col]/df1[col].max()

In [606]:
#Vérification des résultats
df1.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,0.519481,homme,AA,0.70,0.479270,0.0,Normal,0.851485,Non,0.000000,Ascendant,0
1,0.636364,femme,DNA,0.80,0.298507,0.0,Normal,0.772277,Non,0.161290,Plat,1
2,0.480519,homme,AA,0.65,0.469320,0.0,ST,0.485149,Non,0.000000,Ascendant,0
3,0.623377,femme,ASY,0.69,0.354892,0.0,Normal,0.534653,Oui,0.241935,Plat,1
4,0.701299,homme,DNA,0.75,0.323383,0.0,Normal,0.603960,Non,0.000000,Ascendant,0


In [607]:
#Séparer la variable cible (coeur) et les variables explicatives
y = df1['CŒUR']
x = df1.drop('CŒUR', axis=1)

In [608]:
#Subdivision du jeu de données en apprentissage et en test
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size =0.2, random_state =1)

In [609]:
#Vérifier le résultat
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(734, 11) (184, 11) (734,) (184,)


In [610]:
#Vérifier le résultat sur trainset
trainset['CŒUR'].value_counts()

1    398
0    336
Name: CŒUR, dtype: int64

In [611]:
#Vérifier le résultat sur testset
testset['CŒUR'].value_counts()

1    110
0     74
Name: CŒUR, dtype: int64

In [612]:
# creation d'un sous-ensemble
missing_rate = df1.isna().sum()/df1.shape[0]
missing_rate

AGE            0.0
SEXE           0.0
TDT            0.0
PAR            0.0
CHOLESTEROL    0.0
GAJ            0.0
ECG            0.0
FCMAX          0.0
ANGINE         0.0
DEPRESSION     0.0
PENTE          0.0
CŒUR           0.0
dtype: float64

In [613]:
#Encodage des variables qualitatives
for col in df1.select_dtypes('object').columns:
    df1[col] = df1[col].astype('category').cat.codes


In [614]:
# pour retirer les variables manquante
def inputation(df1):
    return df1.dropna(axis=0)

inputation(df1)

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,0.519481,1,0,0.70,0.479270,0.0,1,0.851485,0,0.000000,0,0
1,0.636364,0,3,0.80,0.298507,0.0,1,0.772277,0,0.161290,2,1
2,0.480519,1,0,0.65,0.469320,0.0,2,0.485149,0,0.000000,0,0
3,0.623377,0,1,0.69,0.354892,0.0,1,0.534653,1,0.241935,2,1
4,0.701299,1,3,0.75,0.323383,0.0,1,0.603960,0,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,0.584416,1,2,0.55,0.437811,0.0,1,0.653465,0,0.193548,2,1
914,0.883117,1,1,0.72,0.320066,1.0,1,0.698020,0,0.548387,2,1
915,0.740260,1,1,0.65,0.217247,0.0,1,0.569307,1,0.193548,2,1
916,0.740260,0,0,0.65,0.391376,0.0,0,0.861386,0,0.000000,2,1


In [615]:
#Vérification du type
df1.dtypes.value_counts()

float64    6
int8       5
int64      1
dtype: int64

In [616]:
#Création d'un objet model de la classe DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=0)

In [617]:
#Apprentissage du modèle
modl = model.fit(x_train, y_train)

de evaluation(model):
# model.fit(x_train, y_train)
# ypred = model.predict(x_test)

# print(confusion_matrix(y_test, ypred))
# print(classification_report(y_test, ypred))


ValueError: could not convert string to float: 'homme'

In [ ]:
#Création d'un objet lr de la classe LogisticRegression
#lr = LogisticRegression(solver ='newton-cg', random_state =1)
model = D

NameError: name 'D' is not defined

In [ ]:
#Apprentissage du modèle
model = lr.fit(x_train, y_train)
#model = lr.fit(train_set.drop('CŒUR', axis = 1), train_set['CŒUR'])

In [ ]:
#Probabilité d'appartenance à l'une des classes
predict_proba = model.predict_proba(x_test)
predict_proba[:5,:]

array([[0.39560595, 0.60439405],
       [0.18843348, 0.81156652],
       [0.04970935, 0.95029065],
       [0.09007617, 0.90992383],
       [0.28158313, 0.71841687]])

In [ ]:
#Application du modèle au données de test
y_pred = model.predict(x_test)
y_pred[:20]

array([1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1])

In [ ]:
#matrice de confusion
mc = confusion_matrix(y_test, y_pred)
mc

array([[ 66,   8],
       [  6, 104]])

In [ ]:
#Taux de bonnes prédictions
accuracy = accuracy_score(y_test, y_pred)
accuracy
#Conclusion : le modèle produit 85.32 % de bonne prédiction.

0.9239130434782609

In [ ]:
#Sensibilité
recall = recall_score(y_test, y_pred)
recall
#conclusion : lorsqu'un individu est malade, nous avons 84.11 % de chance que le modèle le prédise.

0.9454545454545454

In [ ]:
#Précision
precision = precision_score(y_test, y_pred)
precision
#Conclusion : lorsque le modèle prédit qu'un individu est malade, nous avons 90 % de chace que cela soit vraie.

0.9285714285714286

In [ ]:
#plus de détails avec classification report
cr = classification_report(y_test, y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.92      0.89      0.90        74
           1       0.93      0.95      0.94       110

    accuracy                           0.92       184
   macro avg       0.92      0.92      0.92       184
weighted avg       0.92      0.92      0.92       184



In [ ]:
#Modèle Trival
df['CŒUR'].value_counts()/df.shape[0]
#Conclusion : si nous nous basons sur le hasard pour faire nos prédiction, nous aurons 55.37 % de bonne prédiction.

1    0.553377
0    0.446623
Name: CŒUR, dtype: float64

In [ ]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données d'entrainement)
model.score(x_train, y_train)

0.8433242506811989

In [ ]:
pickle.dump(lr, open('model.pkl', 'wb'))